In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importing all the necessary libraries
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# Loading the dataset
data = pd.read_csv("/content/drive/MyDrive/synthetic_beverage_sales_data.csv")

In [4]:
print(data.shape)

(8999910, 11)


In [5]:
# Checking the null values
data.isnull().sum()

,0
Order_ID,0
Customer_ID,0
Customer_Type,0
Product,0
Category,0
Unit_Price,0
Quantity,0
Discount,0
Total_Price,0
Region,0


In [6]:
# Checking for any duplicate values
data.duplicated().sum()

0

In [7]:
# Checking number of unique values in column
print(data.nunique())

Order_ID         3000000
Customer_ID        10000
Customer_Type          2
Product               47
Category               4
Unit_Price         12778
Quantity             100
Discount               4
Total_Price       203842
Region                16
Order_Date          1094
dtype: int64


In [8]:
# Checking columns and its datatypes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8999910 entries, 0 to 8999909
Data columns (total 11 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Order_ID       object 
 1   Customer_ID    object 
 2   Customer_Type  object 
 3   Product        object 
 4   Category       object 
 5   Unit_Price     float64
 6   Quantity       int64  
 7   Discount       float64
 8   Total_Price    float64
 9   Region         object 
 10  Order_Date     object 
dtypes: float64(3), int64(1), object(7)
memory usage: 755.3+ MB


In [9]:
# Printing initial 5 rows
print(data.head())

  Order_ID Customer_ID Customer_Type             Product     Category  \
0     ORD1     CUS1496           B2B          Vio Wasser        Water   
1     ORD1     CUS1496           B2B               Evian        Water   
2     ORD1     CUS1496           B2B              Sprite  Soft Drinks   
3     ORD1     CUS1496           B2B  Rauch Multivitamin       Juices   
4     ORD1     CUS1496           B2B        Gerolsteiner        Water   

   Unit_Price  Quantity  Discount  Total_Price             Region  Order_Date  
0        1.66        53      0.10        79.18  Baden-Württemberg  2023-08-23  
1        1.56        90      0.10       126.36  Baden-Württemberg  2023-08-23  
2        1.17        73      0.05        81.14  Baden-Württemberg  2023-08-23  
3        3.22        59      0.10       170.98  Baden-Württemberg  2023-08-23  
4        0.87        35      0.10        27.40  Baden-Württemberg  2023-08-23  


In [10]:
# Drop the ID columns as they are not relevant for model
df = data.drop(columns=['Order_ID', 'Customer_ID'])

In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8999910 entries, 0 to 8999909
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Customer_Type  object 
 1   Product        object 
 2   Category       object 
 3   Unit_Price     float64
 4   Quantity       int64  
 5   Discount       float64
 6   Total_Price    float64
 7   Region         object 
 8   Order_Date     object 
dtypes: float64(3), int64(1), object(5)
memory usage: 618.0+ MB


## Data Cleaning and Feature Engineering


In [12]:
# Customer_Type has only two unique values. So we can apply Label Encoding Here.
le = LabelEncoder()
df['Customer_Type'] = le.fit_transform(df['Customer_Type'])

In [13]:
# As Category column has few unique values we can one-hot encode
df_encoded = pd.get_dummies(df, columns=['Category'], drop_first=True)

In [14]:
# Product and Region columns have many unique values so we use Target Encoding
df_encoded['Product'] = df_encoded.groupby('Product')['Total_Price'].transform('mean')
df_encoded['Region'] = df_encoded.groupby('Region')['Total_Price'].transform('mean')

In [15]:
# Convert 'Order_Date' to datetime format
df_encoded['Order_Date'] = pd.to_datetime(df_encoded['Order_Date'])

# Extract components from the 'Order_Date'
df_encoded['Year'] = df_encoded['Order_Date'].dt.year
df_encoded['Month'] = df_encoded['Order_Date'].dt.month
df_encoded['Day'] = df_encoded['Order_Date'].dt.day
df_encoded['Day_of_Week'] = df_encoded['Order_Date'].dt.dayofweek # 0: Monday, 6: Sunday
df_encoded['Quarter'] = df_encoded['Order_Date'].dt.quarter

In [16]:
# Drop the original 'Order_Date' column after extracting relevant components
data_cleaned = df_encoded.drop(columns=['Order_Date'])

In [17]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8999910 entries, 0 to 8999909
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Customer_Type         int64  
 1   Product               float64
 2   Unit_Price            float64
 3   Quantity              int64  
 4   Discount              float64
 5   Total_Price           float64
 6   Region                float64
 7   Category_Juices       bool   
 8   Category_Soft Drinks  bool   
 9   Category_Water        bool   
 10  Year                  int32  
 11  Month                 int32  
 12  Day                   int32  
 13  Day_of_Week           int32  
 14  Quarter               int32  
dtypes: bool(3), float64(5), int32(5), int64(2)
memory usage: 678.1 MB


In [18]:
# Capping upper limit to 99% as we have seen some outliers at end
percentile_99 = data_cleaned['Total_Price'].quantile(0.99)
data_cleaned['Total_Price'] = np.where(data_cleaned['Total_Price'] > percentile_99, percentile_99, data_cleaned['Total_Price'])

In [19]:
# Applying log transformation to normalize the distribution of 'Total_Price'
# This helps reduce skewness and makes the data more suitable for modeling
data_cleaned['Total_Price'] = np.log1p(data_cleaned['Total_Price'])

In [20]:
data_cleaned.head()

,Customer_Type,Product,Unit_Price,Quantity,Discount,Total_Price,Region,Category_Juices,Category_Soft Drinks,Category_Water,Year,Month,Day,Day_of_Week,Quarter
0,0,30.451677,1.66,53,0.10,4.384274,129.348134,False,False,True,2023,8,23,2,3
1,0,26.845007,1.56,90,0.10,4.847018,129.348134,False,False,True,2023,8,23,2,3
2,0,20.593693,1.17,73,0.05,4.408425,129.348134,False,True,False,2023,8,23,2,3
3,0,56.851696,3.22,59,0.10,5.147378,129.348134,True,False,False,2023,8,23,2,3
4,0,20.954056,0.87,35,0.10,3.346389,129.348134,False,False,True,2023,8,23,2,3


In [21]:
from sklearn.preprocessing import MinMaxScaler

# Applying MinMax Scaling to bring numeric features into a similar range (0 to 1)
# This ensures that features with larger magnitudes do not dominate the model

# Define columns to scale
columns_to_scale = ["Unit_Price", "Quantity", "Discount", "Total_Price"]

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply scaling
data_cleaned[columns_to_scale] = scaler.fit_transform(data_cleaned[columns_to_scale])

# Save the scaler for future use
import pickle
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

# Check the scaled values
data_cleaned.head()


,Customer_Type,Product,Unit_Price,Quantity,Discount,Total_Price,Region,Category_Juices,Category_Soft Drinks,Category_Water,Year,Month,Day,Day_of_Week,Quarter
0,0,30.451677,0.007919,0.525253,0.666667,0.547808,129.348134,False,False,True,2023,8,23,2,3
1,0,26.845007,0.007328,0.898990,0.666667,0.609307,129.348134,False,False,True,2023,8,23,2,3
2,0,20.593693,0.005023,0.727273,0.333333,0.551017,129.348134,False,True,False,2023,8,23,2,3
3,0,56.851696,0.017138,0.585859,0.666667,0.649225,129.348134,True,False,False,2023,8,23,2,3
4,0,20.954056,0.003250,0.343434,0.666667,0.409871,129.348134,False,False,True,2023,8,23,2,3


In [22]:
# Let's split the data into train and test
X = data_cleaned.drop(columns=['Total_Price']) # Features
y = data_cleaned['Total_Price'] # Target variable
# Split the data into 70% train and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42
)
# Check the shape of the resulting datasets
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)

Training data shape: (6299937, 14)
Testing data shape: (2699973, 14)


In [25]:
# Initialize the XGBoost Regressor model with hyperparameters
model = XGBRegressor(
    objective='reg:squarederror',  # Regression objective
    eval_metric='rmse',  # Root Mean Squared Error as evaluation metric
    learning_rate=0.1,  # Controls step size in each boosting iteration
    max_depth=6,  # Maximum depth of decision trees
    n_estimators=100  # Number of boosting rounds (trees)
)

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

In [27]:
X_train.head()

,Customer_Type,Product,Unit_Price,Quantity,Discount,Region,Category_Juices,Category_Soft Drinks,Category_Water,Year,Month,Day,Day_of_Week,Quarter
4924895,1,23.827385,0.004255,0.020202,0.000000,136.528807,False,True,False,2021,5,5,2,2
4719630,1,61.983437,0.017198,0.070707,0.000000,130.964681,True,False,False,2021,11,3,2,4
288530,0,20.663201,0.000768,0.525253,0.333333,125.599861,False,False,True,2023,11,22,2,4
2132060,1,61.983437,0.017789,0.101010,0.000000,138.580030,True,False,False,2022,1,21,4,1
7363775,1,23.723858,0.004964,0.050505,0.000000,124.711312,False,True,False,2021,6,19,5,2


In [26]:
# Predict on training data
y_train_pred = model.predict(X_train)

# Predict on test data
y_test_pred = model.predict(X_test)

# Calculate evaluation metrics for training data
train_mae = mean_absolute_error(y_train, y_train_pred)  # Measures average absolute error
train_mse = mean_squared_error(y_train, y_train_pred)  # Measures squared difference between actual & predicted
train_rmse = np.sqrt(train_mse)  # Square root of MSE for better interpretation

# Calculate evaluation metrics for test data
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)

# Print train vs. test comparison
print("📊 Model Performance Comparison:")
print("----------------------------------")
print(f"🔹 Training Set Metrics:")
print(f"  - MAE:  {train_mae:.6f}")  # Lower is better
print(f"  - MSE:  {train_mse:.6f}")  # Lower is better
print(f"  - RMSE: {train_rmse:.6f}")  # Lower is better
print("----------------------------------")
print(f"🔹 Test Set Metrics:")
print(f"  - MAE:  {test_mae:.6f}")
print(f"  - MSE:  {test_mse:.6f}")
print(f"  - RMSE: {test_rmse:.6f}")

# Interpretation:
# If train and test metrics are close, the model generalizes well.
# If test error is significantly higher than train error, there may be overfitting.

📊 Model Performance Comparison:
----------------------------------
🔹 Training Set Metrics:
  - MAE:  0.001508
  - MSE:  0.000006
  - RMSE: 0.002427
----------------------------------
🔹 Test Set Metrics:
  - MAE:  0.001510
  - MSE:  0.000006
  - RMSE: 0.002433


In [28]:
# Save the model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
